In [ ]:
!pip install pandas openpyxl requests openpyxl face_recognition pillow mediapipe  opencv-python

### **FACE OCCLUSION**

In [ ]:
import os
import pandas as pd
import requests
from urllib.parse import urlparse
from PIL import Image
import face_recognition
import cv2
import mediapipe as mp
import csv
import logging
from datetime import datetime

def process_images_from_excel(input_file, output_dir='Sheets', rows_per_file=1500, log_dir='logs', results_file='results.csv'):
    
    # Configure logging
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, 'image_processing.log')

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler(log_file)
    fh.setLevel(logging.INFO)
    ch = logging.StreamHandler()
    ch.setLevel(logging.ERROR)
    formatter = logging.Formatter('%(asctime)s - Serial Number: %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(ch)

    # Chunk Splitting
    def split_excel_into_files(input_file, output_dir, rows_per_file=1500):
        df = pd.read_excel(input_file, sheet_name='Mar-Image')  # Read only the first sheet
        total_rows = len(df)
        num_files = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file else 0)
        os.makedirs(output_dir, exist_ok=True)

        for i in range(num_files):
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_rows)
            df_chunk = df[start_row:end_row]
            output_file = os.path.join(output_dir, f'Chunk{i+1}.xlsx')
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                df_chunk.to_excel(writer, index=False, sheet_name='Sheet1')
                print(f"Written {len(df_chunk)} rows to {output_file}")

    split_excel_into_files(input_file, output_dir, rows_per_file)
    
    def download_image(url, folder_path):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                parsed_url = urlparse(url)
                image_name = os.path.basename(parsed_url.path)
                image_path = os.path.join(folder_path, image_name)
                with open(image_path, 'wb') as f:
                    f.write(response.content)
                return image_name, None
            else:
                error_message = f"Failed to download (status code: {response.status_code})"
                print(f"{error_message}: {url}")
                return None, error_message
        except requests.exceptions.RequestException as e:
            error_message = f"Error downloading {url}: {e}"
            print(error_message)
            return None, error_message

    # Crop Faces
    def detect_and_crop_faces(image_path, output_dir, expansion_factor=0.3):
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)
        num_faces = len(face_locations)
        if num_faces != 1:
            return image_path, num_faces, None

        pil_image = Image.open(image_path)
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        top, right, bottom, left = face_locations[0]
        width, height = right - left, bottom - top
        expansion_width, expansion_height = int(width * expansion_factor), int(height * expansion_factor)
        new_top, new_bottom = max(0, top - expansion_height), min(image.shape[0], bottom + expansion_height)
        new_left, new_right = max(0, left - expansion_width), min(image.shape[1], right + expansion_width)
        face_image = pil_image.crop((new_left, new_top, new_right, new_bottom))
        face_path = os.path.join(output_dir, f"{name}_face{ext}")
        face_image.save(face_path)
        return face_path, num_faces, None

    def convert_webp_to_jpg(image_path):
        if image_path.endswith('.webp'):
            webp_path = image_path
            jpg_path = os.path.splitext(image_path)[0] + '.jpg'
            with Image.open(webp_path) as img:
                img = img.convert('RGB')
                img.save(jpg_path, 'JPEG')
            os.remove(webp_path)
            return jpg_path
        return image_path

    # FaceMesh
    mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.0,
        min_tracking_confidence=0.90
    )

    def detect_landmarks(image):
        results = mp_face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            return None
        return results.multi_face_landmarks[0]

    def process_image_and_append_results(image_url, csv_writer, serial_number):
        images_dir = 'Images'
        faces_dir = 'Faces'
        os.makedirs(images_dir, exist_ok=True)
        os.makedirs(faces_dir, exist_ok=True)

        image_name, download_error = download_image(image_url, images_dir)
        if download_error:
            csv_writer.writerow([image_name, "Rejected", image_url, download_error])
            return

        image_path = os.path.join(images_dir, image_name)
        face_path, num_faces, face_error = detect_and_crop_faces(image_path, faces_dir)

        if face_error or num_faces != 1:
            error_message = face_error if face_error else "Faces ≠ 1"
            csv_writer.writerow([image_name, "Rejected", image_url, error_message])
            return

        face_path = convert_webp_to_jpg(face_path)

        if face_path.endswith('.jpg') or face_path.endswith('.png'):
            image = cv2.imread(face_path)
            image_top = image[:image.shape[0] // 2, :]
            image_bottom = image[image.shape[0] // 2:, :]
            landmarks_top = detect_landmarks(image_top)
            landmarks_bottom = detect_landmarks(image_bottom)
            top_face_detected = landmarks_top is not None
            bottom_face_detected = landmarks_bottom is not None
            result = 'Accepted' if top_face_detected and bottom_face_detected else 'Rejected'
            error_message = ""
            if result == 'Rejected':
                if not top_face_detected:
                    error_message += "Top Face Error; "
                if not bottom_face_detected:
                    error_message += "Bottom Face Error; "
                error_message = error_message.rstrip("; ")
        else:
            result = 'Rejected'
            error_message = "Invalid image format"

        csv_writer.writerow([image_name, result, image_url, error_message])
        print(f"Image: {image_name}, Result: {result}")
        logging.info(f"{serial_number} - Image: {image_name}, Result: {result}")

    # Process each chunk file
    serial_number = 1
    for chunk_file in os.listdir(output_dir):
        chunk_path = os.path.join(output_dir, chunk_file)
        df_chunk = pd.read_excel(chunk_path, sheet_name='Sheet1')

        with open(results_file, 'a', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            if os.stat(results_file).st_size == 0:
                csv_writer.writerow(['Image Name', 'Result', 'URL', 'Errors'])

            for _, row in df_chunk.iterrows():
                image_url = row[0]
                if pd.notna(image_url):
                    process_image_and_append_results(image_url, csv_writer, serial_number)
                    serial_number += 1

    print("Processing completed. Results saved to results.csv")

if __name__ == "__main__":
    input_file = '/content/drive/MyDrive/ABP/ModelTestingData.xlsx' 
    process_images_from_excel(input_file)

Exception ignored in: <function ZipFile.__del__ at 0x7b4d87961a20>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


Written 1500 rows to Sheets/Chunk1.xlsx
Written 1500 rows to Sheets/Chunk2.xlsx
Written 1500 rows to Sheets/Chunk3.xlsx
Written 1500 rows to Sheets/Chunk4.xlsx
Written 1500 rows to Sheets/Chunk5.xlsx
Written 1500 rows to Sheets/Chunk6.xlsx
Written 1500 rows to Sheets/Chunk7.xlsx
Written 1500 rows to Sheets/Chunk8.xlsx
Written 1500 rows to Sheets/Chunk9.xlsx
Written 1500 rows to Sheets/Chunk10.xlsx
Written 1500 rows to Sheets/Chunk11.xlsx
Written 1500 rows to Sheets/Chunk12.xlsx
Written 1500 rows to Sheets/Chunk13.xlsx
Written 1500 rows to Sheets/Chunk14.xlsx
Written 1500 rows to Sheets/Chunk15.xlsx
Written 1500 rows to Sheets/Chunk16.xlsx
Written 1500 rows to Sheets/Chunk17.xlsx
Written 1500 rows to Sheets/Chunk18.xlsx
Written 1500 rows to Sheets/Chunk19.xlsx
Written 1500 rows to Sheets/Chunk20.xlsx
Written 1500 rows to Sheets/Chunk21.xlsx
Written 1500 rows to Sheets/Chunk22.xlsx
Written 1254 rows to Sheets/Chunk23.xlsx


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
INFO:root:1 - Image: 1709277456400.webp, Result: Rejected


Image: 1709277456400.webp, Result: Rejected


INFO:root:4 - Image: 1707508325394.webp, Result: Rejected


...

INFO:root:4200 - Image: 1707967905166.webp, Result: Accepted


Image: 1707967905166.webp, Result: Accepted


INFO:root:4203 - Image: 1709005687976.webp, Result: Accepted


Image: 1709005687976.webp, Result: Accepted


OSError: cannot write mode RGBA as JPEG

### **HEADWARE**

In [ ]:
!pip install face_recognition pillow pandas requests openpyxl

In [ ]:
# Separating Accepted and Rejected based on Face Occlusion

import pandas as pd

input_file = '/content/drive/MyDrive/ABP/results.csv'
df = pd.read_csv(input_file)

accepted_df = df[df['Result'] == 'Accepted']
rejected_df = df[df['Result'] == 'Rejected']

accepted_file = 'Accepted-Face.csv'
rejected_file = 'Rejected-Face.csv'

accepted_df.to_csv(accepted_file, index=False)
rejected_df.to_csv(rejected_file, index=False)

print(f"Accepted images CSV saved to {accepted_file}")
print(f"Rejected images CSV saved to {rejected_file}")

Accepted images CSV saved to Accepted-Face.csv
Rejected images CSV saved to Rejected-Face.csv


In [ ]:
import os
import pandas as pd
import requests
from urllib.parse import urlparse, urljoin
import face_recognition
from PIL import Image

def process_images(file_path, image_dir='Images', faces_dir='Faces', top_faces_dir='Top_Faces', output_csv_path='Summary.csv'):
    df = pd.read_csv(file_path)

    # Directories to save images and faces
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(faces_dir, exist_ok=True)
    os.makedirs(top_faces_dir, exist_ok=True)

    output_data = []

    def ensure_url_scheme(url):
        parsed_url = urlparse(url)
        if not parsed_url.scheme:
            return urljoin("https://", url)
        return url

    # Download an image from a URL
    def download_image(url, folder_path):
        try:
            url = ensure_url_scheme(url)
            response = requests.get(url)
            if response.status_code == 200:
                parsed_url = urlparse(url)
                image_name = os.path.basename(parsed_url.path)
                image_name = image_name.replace('/', '_').replace(':', '_')
                image_path = os.path.join(folder_path, image_name)
                with open(image_path, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded: {image_name}")
                return image_path, image_name, None
            else:
                error_message = f"Failed to download (status code: {response.status_code}): {url}"
                print(error_message)
                return None, None, error_message
        except requests.exceptions.RequestException as e:
            error_message = f"Error downloading {url}: {e}"
            print(error_message)
            return None, None, error_message

    # Crop Faces and Save the Top Faces
    def crop_faces(image_path, output_dir, top_faces_dir, original_url, expansion_factor=0.5):
        try:
            image = face_recognition.load_image_file(image_path)
            face_locations = face_recognition.face_locations(image)
            if len(face_locations) != 1:
                return 0, "Faces ≠ 1"

            pil_image = Image.open(image_path)
            url_safe_name = original_url.replace('/', '_').replace(':', '_')

            for i, (top, right, bottom, left) in enumerate(face_locations):
                width = right - left
                height = bottom - top

                expansion_width = int(width * expansion_factor)
                expansion_height = int(height * expansion_factor)

                new_top = max(0, top - expansion_height)
                new_bottom = min(image.shape[0], bottom + expansion_height)
                new_left = max(0, left - expansion_width)
                new_right = min(image.shape[1], right + expansion_width)

                # Crop the face
                face_image = pil_image.crop((new_left, new_top, new_right, new_bottom))
                face_path = os.path.join(output_dir, f"{url_safe_name}_face_{i+1}.jpg")
                face_image.save(face_path)
                # print(f"Saved face {i+1} from {url_safe_name}")

                # Crop and save the top half of the face
                top_half_face_image = face_image.crop((0, 0, face_image.width, face_image.height // 2))
                top_half_face_path = os.path.join(top_faces_dir, f"{url_safe_name}.jpg")
                top_half_face_image.save(top_half_face_path)
                # print(f"Saved top half face {i+1} from {url_safe_name}")

            return len(face_locations), None
        except Exception as e:
            error_message = f"Error processing {image_path}: {e}"
            print(error_message)
            return 0, error_message

    # Iterate through each URL
    for index, row in df.iterrows():
        url = row['URL']
        image_name_from_file = row['Image Name']
        if pd.notna(url):                                                           # NaN
            image_path, image_name, download_error = download_image(url, image_dir)
            if image_path:
                num_faces, process_error = crop_faces(image_path, faces_dir, top_faces_dir, url)
                if num_faces != 1:
                    result = "Rejected"
                    process_error = "Faces ≠ 1"
                else:
                    result = ""
            else:
                result = "Rejected"
                process_error = download_error
            output_data.append({
                "Image Name": image_name if image_name else image_name_from_file,
                "URL": url,
                "Result": result,
                "Error": process_error if process_error else "None"
            })

    # Save the output data to a CSV file
    output_df = pd.DataFrame(output_data)
    output_df.to_csv(output_csv_path, index=False)
    print(f"Output saved to {output_csv_path}")

if __name__ == "__main__":
    file_path = '/content/drive/MyDrive/ABP/Accepted-Face.csv'  # Replace with your actual file path
    process_images(file_path)

Downloaded: 1709047815520.webp
Downloaded: 1707460761789.webp
Downloaded: 1710990253588.webp
Downloaded: 1708069980972.webp
Error processing Images/1708069980972.webp: cannot write mode RGBA as JPEG
Downloaded: 1707145646321.webp
Downloaded: 1709047815520.webp
Downloaded: 1707460761789.webp
Downloaded: 1710990253588.webp
Downloaded: 1708069980972.webp
Error processing Images/1708069980972.webp: cannot write mode RGBA as JPEG
Downloaded: 1707145646321.webp
Downloaded: 1706888700234.webp
Downloaded: 1707666823254.webp
Downloaded: 1706786196391.webp
Downloaded: 1709218035065.webp
Downloaded: 1707381364199.webp
Downloaded: 1706959497670.webp
Downloaded: 1707223529384.webp
Downloaded: 1709551923361.webp
Downloaded: 1707924674222.webp
Downloaded: 1709058153620.webp
Downloaded: 321917161551604399014.jpg
Downloaded: 1711264047091.webp
Downloaded: 1708185029115.webp
Downloaded: 1707468283506.webp
Downloaded: 1707060832441.webp
Downloaded: 1707761806735.webp
Downloaded: 1708545255863.webp
Downlo

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python detect.py --source '/content/drive/MyDrive/ABP/Top_Faces' --img 512 --device 0 --weights '/content/drive/MyDrive/ABP/best.pt' --conf-thres 0.10 --iou-thres 0.35 --save-conf --name '/content/drive/MyDrive/ABP/Results' --save-txt

detect: weights=['/content/drive/MyDrive/ABP/best.pt'], source=/content/drive/MyDrive/ABP/Top_Faces, data=data/coco128.yaml, imgsz=[512, 512], conf_thres=0.1, iou_thres=0.35, max_det=1000, device=0, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=/content/drive/MyDrive/ABP/Results, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50705208 parameters, 0 gradients, 236.7 GFLOPs
image 1/1706 /content/drive/MyDrive/ABP/Top_Faces/https___cdn.abpweddings.com_documents_001243144b6bb64a6b5c46246504d8fd_1707540434056.webp?width=150?width=150.jpg: 256x512 (no detections), 133.8ms
image 2/1706 /content/drive/MyDrive/ABP/Top_Faces/https___cdn.abpweddings.com_documents_00245d6

In [ ]:
# HeadWare Summary

import os
import pandas as pd

results_dir = '/content/drive/MyDrive/ABP/Results/labels'
source_dir = '/content/drive/MyDrive/ABP/Top_Faces'

label_map = {
    '0': 'Hat',
    '1': 'Helmet',
    '2': 'None',
    '3': 'hijab'
}

output_data = []
image_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]

for image_file in image_files:
    image_name = os.path.splitext(image_file)[0]
    txt_file_path = os.path.join(results_dir, f"{image_name}.txt")

    # Check if the corresponding text file exists
    if os.path.exists(txt_file_path):
        with open(txt_file_path, 'r') as file:
            lines = file.readlines()
            if lines:
                classes_detected = [label_map.get(line.split()[0], 'unknown') for line in lines]  # Replace ID with name
                classes_detected = ", ".join(classes_detected)
            else:
                classes_detected = "no detections"
    else:
        classes_detected = "no detections"

    # Append to Output Data
    output_data.append({
        "Image Name": image_name,
        "Detected Classes": classes_detected
    })

output_df = pd.DataFrame(output_data)

# Save Output
output_csv_path = '/content/drive/MyDrive/ABP/HeadWare-Detection.csv'  # Adjust the path as needed
output_df.to_csv(output_csv_path, index=False)
print(f"Output saved to {output_csv_path}")

Output saved to /content/drive/MyDrive/ABP/HeadWare-Detection.csv


In [ ]:
# Formatting HeadWare Summary

import os
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlunparse

summary_df = pd.read_csv('/content/drive/MyDrive/ABP/Summary.csv')
headware_df = pd.read_csv('/content/drive/MyDrive/ABP/HeadWare-Detection.csv')

def normalize_url(url):
    parsed_url = urlparse(url)
    return urlunparse(parsed_url._replace(query=''))

headware_df['Image Name'] = headware_df['Image Name'].apply(lambda x: x.split('_')[-1])

summary_df['Normalized URL'] = summary_df['URL'].apply(normalize_url)
headware_df['Normalized URL'] = headware_df['Image Name'].apply(normalize_url)

headware_dict = headware_df.set_index('Image Name')['Detected Classes'].to_dict()

summary_df['Result'] = ''
summary_df['Error'] = ''

for index, row in summary_df.iterrows():
    image_name = row['Image Name']
    if image_name in headware_dict:
        detected_classes = headware_dict[image_name]
        if detected_classes == 'no detections' or detected_classes == 'None':
            summary_df.at[index, 'Result'] = 'Accepted'
        else:
            summary_df.at[index, 'Result'] = 'Rejected'
            summary_df.at[index, 'Error'] = 'Head Ware Present'
    else:
        summary_df.at[index, 'Result'] = 'Accepted'

# Filtering Columns
output_df = summary_df[['Image Name', 'Result', 'URL', 'Error']]

output_csv_path = '/content/drive/MyDrive/ABP/HeadWare-Detection.csv'
output_df.to_csv(output_csv_path, index=False)
print(f"Output saved to {output_csv_path}")

Output saved to /content/drive/MyDrive/ABP/HeadWare-Detection.csv


### **MERGING**

In [ ]:
import pandas as pd

df1 = pd.read_csv("/content/drive/MyDrive/ABP/HeadWare-Detection.csv")
df2 = pd.read_csv("/content/drive/MyDrive/ABP/Rejected-Face.csv")

df2 = df2.rename(columns={"Errors": "Error"})

df_merged = pd.concat([df1, df2], ignore_index=True)

# print(df_merged)  # This will display the merged DataFrame
df_merged.to_csv("/content/drive/MyDrive/ABP/Model-Result.csv", index=False)  # This will save the merged DataFrame to a new CSV